In [1]:
from numpy import sum, sqrt, sort, argsort, zeros, arange, array, isclose, max
from numpy.random import randint, rand
from subprocess import call
import matplotlib.pyplot as plt

import numpy as np

In [2]:
distance = lambda X,Y :sqrt(
    sum(Y**2,axis=1, keepdims=True) - 2 * Y@X.T + sum(X**2,axis=1,keepdims=True).T
)

In [3]:
def write_input(X,Y):
    n,d = X.shape
    m,d = Y.shape
    with open("data.in", 'w') as file:
        # Write the dimetions in the file
        file.write("%d %d %d\n"%(n,m,d))
    
        #Write X in the file
        for row in X:
            for item in row:
                file.write("%g "%item)
            file.write("\n")
    
        #Write X in the file
        for row in Y:
            for item in row:
                file.write("%g "%item)
            file.write("\n")
    return

In [4]:
def read_input():
    with open("data.in", "r") as file:
        # Read the dimentions
        first_line = file.readline()
        n,m,d = list(map(int, first_line.strip().split()))
        
        # Read the corpus
        X = zeros((n,d))
        for i in range(n):
            next_line = file.readline()
            X[i,:] = list(map(float, next_line.strip().split()))
        
        # Read the distances
        Y = zeros((m,d))
        for i in range(m):
            next_line = file.readline()
            Y[i,:] = list(map(float, next_line.strip().split()))
    
    return n,m,d,X,Y

In [5]:
def run (k):
    call(['./main_sequential', str(k)])

In [6]:
def kNN(X,Y,k):
    # sqrt( sum(X.^2,2) - 2*X*Y' + sum(Y.^2,2)' )
    D = sqrt(
            # sum(X.^2,2) - 2*X*Y' + sum(Y.^2,2)'
            sum(Y**2,axis=1, keepdims=True) - 2 * Y@X.T + sum(X**2,axis=1,keepdims=True).T
            )
    
    m,n   =  D.shape
    nidx  =  zeros((m,k), dtype=int)
    ndist =  zeros((m,k))
    
    for qp in range(m):
        nidx[qp,:]  =  argsort(D[qp,:])[:k]
        ndist[qp,:] =  sort(D[qp,:])[:k]
        
    return nidx, ndist, m, k

In [12]:
def read_output():
    
    
    with open("data.out", "r") as file:
        # Read the dimentions
        first_line = file.readline()
        m,k = list(map(int, first_line.strip().split()))
        
        # Read the indexes
        nidx = zeros((m,k), dtype=int)
        for i in range(m):
            next_line = file.readline()
            nidx[i,:] = list(map(float, next_line.strip().split()))
        
        # Read the distances
        ndist = zeros((m,k))
        for i in range(m):
            next_line = file.readline()
            ndist[i,:] = list(map(float, next_line.strip().split()))
    
    return nidx, ndist, m, k

In [106]:
def test(X,Y,k):
    
    response = {True: 'CORRECT', False: 'WRONG'}
    
    res = True
    
    # Run the program
    write_input(X,Y)
    run(k)
    nidx,ndist,m,k = read_output()
    
    # Check that distances are correct
    D = distance(X,Y)
    
    # (i) Checking that reported distances are correct
    res = True
    for i,d in enumerate(D):
        res = res and isclose(ndist[i], d[nidx[i,:]]).all()
        
    # (ii)  Validating that distances are sorted in non-decreasing order  
    res = res and (ndist[:, :-1] < ndist[:,1:]).all()
    
    # (iii) Ensuring there are no other points closer than the kth neighbor
    maxDist = max(ndist, axis=1)
    minDist = zeros(m)
    for i,dist in enumerate(D):
        minDist[i] = min([ d for j,d in enumerate(dist) if j not in nidx[i] ])
    minDist.shape
    res = res and (minDist > maxDist).all()
    
    return response[res]

In [95]:
n,m,d,X,Y = read_input()
k = 13

In [96]:
# Run the program
write_input(X,Y)
run(k)
nidx,ndist,m,k = read_output()

In [108]:
test(X,Y,k)

'CORRECT'